In [1]:
import os

import numpy as np
import pandas as pd
import cv2
from PIL import Image
import PIL.ImageOps
from sklearn.model_selection import train_test_split
from sklearn import datasets
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns

import ray 
import cv2

from bs4 import BeautifulSoup as BSHTML
import urllib
import requests

In [2]:
ray.shutdown()
ray.init()

2020-03-10 15:20:59,069	INFO resource_spec.py:216 -- Starting Ray with 4.79 GiB memory available for workers and up to 2.4 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '129.69.205.59',
 'redis_address': '129.69.205.59:26651',
 'object_store_address': '/tmp/ray/session_2020-03-10_15-20-59_059311_33204/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-10_15-20-59_059311_33204/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-03-10_15-20-59_059311_33204'}

In [4]:
files = os.listdir()

@ray.remote
def flatten_images(path, number=50):
    if not os.path.exists(path):
        return 1
    
    print(path)
    print()
    
    if os.path.exists(path.split('/')[0]+'/flat_img_{}.csv'.format(number)):
        print('Skipping....')
        return 
    images = []
    names = []
    for i,ff in enumerate(os.listdir(path)):
        if ff.startswith('.'):
            print('not an image: ' + ff)
            continue
        if i <2:
            print(ff.split('.')[0])
            
        img = cv2.imread(path + ff, cv2.IMREAD_UNCHANGED)
        if img is None:
            print('!!!! '+ ff)
            continue
        if len(img.shape) > 2 and img.shape[2] == 4:
            #convert the image from RGBA2RGB
            try:
                img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
            except:
                continue

        img = pd.DataFrame(np.array(img).flatten()).T
        #print(img)
        images.append(img)
        names.append(ff.split('.')[0])

    print(len(images))
    if len(images)>0:
        imgs = pd.concat(images, axis=0)
        imgs['filename'] = names
        imgs['labels'] = ['_'.join(n.split('_')[:-1]) for n in names]
        #print(imgs.head())
        imgs = imgs.dropna(axis=1)

        
        imgs.to_csv(path.split('/')[0]+'/flat_img_{}.csv'.format(number),index=False)
        print(imgs.head(2))
    print('----Finished: {}----'.format(path))

    return 0
    


for number in [50,200]:
    for f in files:
        if os.path.isdir(f) and not f.startswith('.') and not f.startswith('_') and not f.startswith('MNIST') and not f.startswith('textures'):
            print(f)
            if number == 50:
                path = f+'/thumbnails{}/'.format('')
            else:
                path = f+'/thumbnails{}/'.format(number)
            if not os.path.exists(path):
                print('x')
                continue
            else:
                flatten_images.remote(path, number)
        
        

dogs
fashionmnist
pets
oxford_buildings
flowers
paintings
cinic10
caltech
paris_buildings
coil-100
dogs
fashionmnist
pets
oxford_buildings
flowers
paintings
cinic10
caltech
paris_buildings
coil-100
(pid=33221) fashionmnist/thumbnails/
(pid=33221) 
(pid=33221) Skipping....
(pid=33224) pets/thumbnails/
(pid=33224) 
(pid=33224) Skipping....
(pid=33224) fashionmnist/thumbnails200/
(pid=33224) 
(pid=33224) 0
(pid=33224) ----Finished: fashionmnist/thumbnails200/----
(pid=33224) paintings/thumbnails200/
(pid=33224) 
(pid=33224) Skipping....
(pid=33224) caltech/thumbnails200/
(pid=33224) 
(pid=33224) Skipping....
(pid=33224) coil-100/thumbnails200/
(pid=33224) 
(pid=33227) cinic10/thumbnails/
(pid=33227) 
(pid=33227) Skipping....
(pid=33227) dogs/thumbnails200/
(pid=33227) 
(pid=33227) Skipping....
(pid=33227) pets/thumbnails200/
(pid=33227) 
(pid=33227) Skipping....
(pid=33227) oxford_buildings/thumbnails200/
(pid=33227) 
(pid=33227) Skipping....
(pid=33227) flowers/thumbnails200/
(pid=33227)

In [3]:
files = os.listdir()
number = 50

@ray.remote
def reformat(f, number):
    print('Started: {}'.format(f))
    path = f+'/flat_img_{}.csv'.format(number)
    try:
        df = pd.read_csv(path)
    except:
        return
    #df['filename'] = df['labels']
    df['labels'] = ['_'.join(row['filename'].split('_')[:-1]) for i,row in df.iterrows()]
    df.to_csv(f+'/flat_img_{}_1.csv'.format(number), index=False)
    print('Finished: {}'.format(f))
    print(df.head(2))
    return 1

for number in [50,200]:
    for f in files:
        if os.path.isdir(f) and not f.startswith('.') and not f.startswith('_') and not f.startswith('MNIST') and not f.startswith('textures'):
            reformat.remote(f, number)
            

(pid=33225) Started: dogs
(pid=33218) Started: oxford_buildings
(pid=33221) Started: flowers
(pid=33219) Started: pets
(pid=33220) Started: fashionmnist
(pid=33224) Started: cinic10
(pid=33223) Started: caltech
(pid=33222) Started: paintings
(pid=33222) Finished: paintings
(pid=33222)      0    1    2    3    4    5  ...  7496  7497  7498  7499  filename  labels
(pid=33222) 0  149  177  160  145  166  154  ...    45    16    15    39   boat_94    boat
(pid=33222) 1   86   85   90   90   89   94  ...   109   101   100   109   boat_57    boat
(pid=33222) 
(pid=33222) [2 rows x 7502 columns]
(pid=33227) Started: paris_buildings
(pid=33218) Finished: oxford_buildings
(pid=33218)      0    1   2    3  ...  7498  7499                 filename            labels
(pid=33218) 0  153   94  44  159  ...   161   163         ashmolean_000007         ashmolean
(pid=33218) 1  221  136  66  221  ...    30    29  radcliffe_camera_000015  radcliffe_camera
(pid=33218) 
(pid=33218) [2 rows x 7502 columns]
